In [1]:
import numpy as np
import copy

In [2]:
population = np.load('population.npy')
num_node = 64
num_stay = 5
time_length = 100
num_people = 50

In [3]:
def generate_node(num_node, node_length = 1):
    # assume map is grid and distance between adjacent node is 1
    num_side = int(np.sqrt(num_node))
    nodes = np.zeros((num_node, 2))
    for i in range(num_side):
        for j in range(num_side):
            index = i + j * num_side
            nodes[index, 0] = i * node_length + 1
            nodes[index, 1] = j * node_length + 1
    return nodes

In [4]:
def generate_stay(num_stay, num_node):
    # generate stay location not to be adjacent to each other
    adjacent = adjacent_node(num_node)
    result = np.zeros(num_stay)
    choice_set = [x for x in range(num_node)]
    for i in range(num_stay):
        if i == 0:
            result[i] = np.random.choice(choice_set, 1)
        else:
            choice_set = list(set(choice_set) - set(adjacent[int(result[i - 1])]))
            result[i] = np.random.choice(choice_set, 1)
    return result
            

In [5]:
def adjacent_node(num_node):
    # find adjacent node which can be approached within 1 time step ( assume everyone velocity is constant)
    nodes = generate_node(num_node)
    threshold = np.sqrt(3)
    result = [];
    for i in range(num_node):
        distance = np.zeros((num_node, 1))
        for j in range(num_node):
            distance[j, 0] = np.sqrt((nodes[i, 0] - nodes[j, 0])**2 + (nodes[i, 1] - nodes[j, 1])**2)
        index = np.where(distance <= threshold)[0]
        result.append(index)
    return result

In [6]:
def generate_route(origin, destination, num_node):
    # given origin and destination generate route according to the distance to destination
    origin = int(origin)
    destination = int(destination)
    nodes = generate_node(num_node)
    adjacent = adjacent_node(num_node)
    route = []
    current = origin
    route.append(current)
    while current != destination:
        if len(route) == 1:
            possible_set = list(adjacent[current])
        else:
            possible_set = list(adjacent[current])
            possible_set = list(set(possible_set) - set(route[-2:]))
        
        probability = []
        for i in range(len(possible_set)):
            d = np.sqrt((nodes[possible_set[i], 0] - nodes[destination, 0])**2 + (nodes[possible_set[i], 1] - nodes[destination, 1])**2)
            probability.append(np.exp(- d **2))
        probability = probability/sum(probability)
        next_step = int(np.random.choice(possible_set, 1, p = probability))
        route.append(next_step)
        current = next_step
    return route
            

In [7]:
def generate_trajectory(time_length, num_node, num_stay):
    #generate the trajectory from the given number of stay
    nodes = generate_node(num_node)
    adjacent = adjacent_node(num_node)
    stays = generate_stay(num_stay, num_node)
    stays = np.insert(stays, len(stays), stays[0])
    
    trajectory = np.zeros(time_length)
    total_route = []
    route_time = 0
    for i in range(num_stay):
        if i == num_stay - 1:
            route = generate_route(stays[i], stays[0], num_node)
        else:
            route = generate_route(stays[i], stays[i + 1], num_node)
        route_time = route_time + len(route)
        total_route.append(route)
    left_time = time_length - route_time
    
    stay_time = np.zeros(num_stay + 1)
    stay_time[0] = int(np.floor(left_time * 0.4))
    stay_time[1] = int(np.floor(left_time * 0.4))
    left_time = left_time - stay_time[0] - stay_time[1]
    probability = np.random.uniform(low = num_stay/(left_time - 1), high = 1, size = num_stay - 1)
    probability = probability / np.sum(probability)
    
    for i in range(2, num_stay + 1):
        stay_time[i] = int(np.floor(left_time * probability[i - 2]))
    remain_time = left_time - np.sum(stay_time[2:])
    stay_time[0] = stay_time[0] + remain_time
    
    index = 0
    for i in range(num_stay):
        trajectory[index : int(stay_time[i]) + index] = stays[i] * np.ones(int(stay_time[i]))
        index = index + int(stay_time[i])
        route = total_route[i]
        trajectory[index : index + len(route)] = route
        index = index + len(route)
    trajectory[index : int(stay_time[i + 1]) + index] = stays[0] * np.ones(int(stay_time[i + 1]) )
    return trajectory, stays, stay_time

In [8]:
def generate_observation_probability(num_node, population):
    # generate observation probability (relationship between observed data and actual data)
    # row is the actual state, column is the observed data
    nodes = generate_node(num_node)
    result = np.zeros((num_node, num_node))
    coeff_min = 1
    coeff_max = 5
    max_population = np.max(population)
    min_population = np.min(population)
    population = (population - min_population) / (max_population - min_population)
    
    for i in range(num_node):
        for j in range(num_node):
            distance = np.sqrt((nodes[i, 0] - nodes[j, 0])**2 + (nodes[i, 1] - nodes[j, 1])**2)
            coeff = - (coeff_max - coeff_min) * population[i] + coeff_max
            result[i, j] = np.exp(- coeff * distance ** 2)
        result[i, :] = result[i, :]/ np.sum(result[i, :])
    return result

In [9]:
def generate_observed(num_node, trajectory, population):
    #generate cdr with localization error
    choice_set = [x for x in range(num_node)]
    
    result = np.zeros(len(trajectory))
    for i in range(len(trajectory)):
        probability = generate_observation_probability(num_node, population[:, i])
        choice_prob = probability[int(trajectory[i]), :]
        result[i] =  np.random.choice(choice_set, 1, p = choice_prob)
    return result

In [10]:
def generate_multi_observed(num_node, trajectory, population, percentage):
    #generate cdr with oscillation
    choice_set = [x for x in range(num_node)]
    time_length = len(trajectory)
    num_multi = int(time_length * percentage)
    
    result = []
    for i in range(len(trajectory)):
        probability = generate_observation_probability(num_node, population[:, i])
        choice_prob = probability[int(trajectory[i]), :]
        
        multi_point = np.random.randint(0, time_length - 1, (1, num_multi))
        if np.isin(i, multi_point):
            temp_num = np.random.randint(2, 6)
            temp_result = np.random.choice(choice_set, temp_num, p = choice_prob)
            temp_result = temp_result.tolist()
            result.append(temp_result)
        else:
            temp_result = np.random.choice(choice_set, 1, p = choice_prob)
            temp_result = temp_result.tolist()
            result.append(temp_result)
        
    return result

In [11]:
def extract_od(stay, num_node):
    #row : origin, column :destination
    stay = list(stay)
    num_row = len(stay)
    result = np.zeros((num_node, num_node))
    for i in range(num_row):
        person = stay[i]
        person = person.astype(int)
        for j in range(len(person) - 1):
            if person[j] != person[j + 1]:
                result[person[j], person[j + 1]] = result[person[j], person[j + 1]] + 1
        result[person[j + 1], person[0]] = result[person[j + 1], person[0]] + 1
    
    return result

In [12]:
def organize_stay_time(stay, stay_time, num_node):
    stay = list(stay)
    result = []
    num_row = len(stay)
    for i in range(num_node):
        result.append([])
    for i in range(num_row):
        person_stay = stay[i].astype(int)
        person_stay_time = stay_time[i].astype(int)
        for j in range(len(person_stay)):
            result[person_stay[j]].append(person_stay_time[j])
    return result
        

In [13]:
A = np.load('A2.npy')
C = np.load('C2.npy')

In [13]:
def find_nearest(point, num_node):
    nodes = generate_node(num_node)
    distance = np.zeros(num_node)
    for i in range(num_node):
        distance[i] = np.sqrt((point[0] - nodes[i, 0])**2 + (point[1] - nodes[i, 1])**2)
    index = np.argmin(distance)
    return index

In [14]:
def missing_observed(observations_test, percentage):
    #generate cdr with irregularity
    num_people = len(observations_test)
    time_length = len(observations_test[0])
    num_missing = int(percentage * time_length)
    
    result = []
    for i in range(num_people):
        result.append([])
        missing_point = np.random.randint(0, time_length - 1, (1, num_missing))
        for j in range(time_length):
            if np.isin(j, missing_point):
                result[i].append([])
            else:
                result[i].append(observations_test[i][j])
    return result

In [15]:
def alpha_foward(observation, A, C, num_node, time_length):
    #hmm viterbi forward procedure
    result = np.zeros((num_node, time_length))
    epsilon = 10 ** (- 10)
    for j in range(time_length):
        for i in range(num_node):
            if j == 0:
                temp = 1
                for k in range(len(observation[j])):
                    temp = temp * C[i, observation[j][k]]
                result[i, j] = temp
            else:
                temp = 1
                for k in range(len(observation[j])):
                    temp = temp * C[i, observation[j][k]]
                    
                for k in range(num_node):
                    result[i, j] = result[i, j] + result[k , j - 1] * A[k, i] * temp
    return result

In [16]:
def beta_backward(observation, A, C, num_node, time_length):
    #hmm viterbi backward procedure
    result = np.zeros((num_node, time_length))
    for j in range(time_length - 1, - 1, - 1):
        for i in range(num_node):
            if j == time_length - 1:
                result[i, j] = 1
            else:
                for k in range(num_node):
                    temp = 1
                    for l in range (len(observation[j + 1])):
                        temp = temp * C[k, observation[j + 1][l]]
                    result[i, j] = result[i, j] + result[k, j + 1] * A[i, k] * temp

    return result

In [17]:
def real_viterbi(observations_test, A, C, num_node):
    #modified viterbi for cdr(with irregular and oscillation)
    num_people = len(observations_test)
    time_length = len(observations_test[0])
    result = np.zeros((num_people, time_length))
    adjacent = adjacent_node(num_node)
    epsilon = 10 **(-10)
    
    for i in range(num_people):
        person_obs = copy.deepcopy(observations_test[i])
        forward_value = alpha_foward(person_obs, A, C, num_node, time_length)
        backward_value = beta_backward(person_obs, A, C, num_node, time_length)
        Baum_welch = np.multiply(forward_value, backward_value)
        cost_result = np.zeros((time_length, num_node))
        prev_step = np.zeros((time_length, num_node))
        
        for j in range(time_length):
            if len(person_obs[j]) == 0:
                person_obs[j] = np.array([np.argmax(Baum_welch[:, j])])
                
            if j == 0:
                for k in range(num_node):
                    temp = 1
                    for m in range(len(person_obs[j])):
                        temp = temp * C[k, person_obs[j][m]]
                    cost_result[j, k] = np.log(temp + epsilon)
            else:
                for k in range(num_node):
                    temp = 1
                    for m in range(len(person_obs[j])):
                        temp = temp * C[k, person_obs[j][m]]
                    possible_choice = adjacent[k]
                    possible_cost = np.zeros(len(possible_choice))
                    for l in range(len(possible_choice)):
                        possible_cost[l] = cost_result[j - 1, possible_choice[l]] + np.log(A[possible_choice[l], k] + epsilon) + np.log(temp + epsilon)
                    max_val = np.max(possible_cost)
                    max_index = np.argmax(possible_cost)
                    cost_result[j, k] = max_val
                    prev_step[j, k] = possible_choice[max_index]
        
        person_traj = np.zeros(time_length)
        max_val = np.max(cost_result[time_length - 1, :])
        max_index = np.argmax(cost_result[time_length - 1, :])
        prev_step = prev_step.astype(int)
        
        person_traj[time_length - 1] = max_index
        for j in range(time_length - 1):
            person_traj[time_length - 2 - j] = prev_step[time_length - 1 - j, int(person_traj[time_length - 1 - j])]
        
        result[i, :] = person_traj
        
    return result

In [18]:
def extract_current(obs, num_node):
    #find most likely current state
    count = np.bincount(obs)
    index = np.where(count == np.max(count))
    if len(index) == 1:
        result = np.bincount(obs).argmax()
    else:
        nodes = generate_node(num_node)
        point = np.zeros(2)
        for i in range(len(obs)):
            point[0] = point[0] + nodes[obs[i], 0]
            point[1] = point[1] + nodes[obs[i], 1]
        point = point/len(obs)
        
        result = find_nearest(point, num_node)
    return result

In [19]:
def max_distance(prev, current, num_node):
    nodes = generate_node(num_node)
    current = int(current)
    prev_size = len(prev)
    distance = 0
    if prev_size == 0:
        distance = 0
    else:
        for i in range(prev_size):
            index = int(prev[i])
            temp_distance = np.sqrt((nodes[current, 0] - nodes[index, 0])**2 + (nodes[current, 1] - nodes[index, 1])**2)
            if distance < temp_distance:
                distance = temp_distance
    return distance

In [20]:
def extract_stay(result, spatial_threshold, temporal_threshold):
    #clustering method for staying location
    num_people = result.shape[0]
    time_length = result.shape[1]
    stays = []
    arrive_times = []
    depart_times = []
    #cluster locations
    for i in range(num_people):
        person_result = result[i, :]
        stay = []
        arrive_time = [0]
        depart_time = []
        prev = []
        for j in range(time_length):
            current = person_result[j]
            if max_distance(prev, current, num_node) > spatial_threshold:
                possible_location = np.bincount(prev).argmax()
                stay.append(possible_location)
                arrive_time.append(j)
                if j != 0:
                    depart_time.append(j - 1)
                prev = []
                    
            prev.append(current)
        possible_location = np.bincount(prev).argmax()
        stay.append(possible_location)
        depart_time.append(j)
    
        #filter out small duration
        index_list = []
        for j in range(len(stay)):
            if depart_time[j] - arrive_time[j] + 1> temporal_threshold:
                index_list.append(j)
    
        stay = np.asarray(stay)
        arrive_time = np.asarray(arrive_time)
        depart_time = np.asarray(depart_time)
    
        stay = stay[index_list]
        arrive_time = arrive_time[index_list]
        depart_time = depart_time[index_list]
        
        stays.append(stay)
        arrive_times.append(arrive_time)
        depart_times.append(depart_time)
    return stays, arrive_times, depart_times

In [21]:
def extract_stay_obs(observations, spatial_threshold, temporal_threshold):
    #clustering method for staying location
    num_people = len(observations)
    time_length = len(observations[0])
    stays = []
    arrive_times = []
    depart_times = []
    #cluster locations
    for i in range(num_people):
        person_result = observations[i]
        stay = []
        arrive_time = [0]
        depart_time = []
        prev = []
        for j in range(time_length):
            if len(person_result[j]) > 0:
                current = extract_current(person_result[j], num_node)
                if max_distance(prev, current, num_node) > spatial_threshold:
                    possible_location = np.bincount(prev).argmax()
                    stay.append(possible_location)
                    arrive_time.append(j)
#                    arrive_time.append(j - len(prev))
                    if j != 0:
                        depart_time.append(j - 1)
                    prev = []
                    
                prev.append(current)
        possible_location = np.bincount(prev).argmax()
        stay.append(possible_location)
#        arrive_time.append(j - len(prev) + 1)
        depart_time.append(j)
    
        #filter out small duration
        index_list = []
        for j in range(len(stay)):
            if depart_time[j] - arrive_time[j] + 1> temporal_threshold:
                index_list.append(j)
    
        stay = np.asarray(stay)
        arrive_time = np.asarray(arrive_time)
        depart_time = np.asarray(depart_time)
    
        stay = stay[index_list]
        arrive_time = arrive_time[index_list]
        depart_time = depart_time[index_list]
        
        stays.append(stay)
        arrive_times.append(arrive_time)
        depart_times.append(depart_time)
    return stays, arrive_times, depart_times

In [22]:
def extract_od(stay, num_node):
    #row : origin, column :destination
    stay = list(stay)
    num_row = len(stay)
    result = np.zeros((num_node, num_node))
    for i in range(num_row):
        person = stay[i]
        person = person.astype(int)
        for j in range(len(person) - 1):
            if person[j] != person[j + 1]:
                result[person[j], person[j + 1]] = result[person[j], person[j + 1]] + 1
    
    return result

In [23]:
def percentage_error(od_result, od_truth, num_node):
    #error analysis
    total_num = np.sum(np.sum(od_truth))
    count = 0
    for i in range(num_node):
        for j in range(num_node):
            if od_result[i, j] >= od_truth[i, j]:
                count = count + od_truth[i, j]
            else:
                count = count + od_result[i, j]
    return count/total_num
            

In [24]:
def extract_stay_modified(result, spatial_threshold, temporal_threshold, recalculate_threshold):
    #modified clustering method with reconsideration
    num_people = result.shape[0]
    time_length = result.shape[1]
    stays = []
    arrive_times = []
    depart_times = []
    #cluster locations
    for i in range(num_people):
        person_result = result[i, :]
        stay = []
        arrive_time = [0]
        depart_time = []
        prev = []
        for j in range(time_length):
            current = person_result[j]
            if current >= 0:
                if max_distance(prev, current, num_node) > spatial_threshold:
                    possible_location = np.bincount(prev).argmax()
                    stay.append(possible_location)
                    arrive_time.append(j)
                    if j != 0:
                        depart_time.append(j - 1)
                    prev = []
                    
                prev.append(current)
        possible_location = np.bincount(prev).argmax()
        stay.append(possible_location)
        depart_time.append(j)
    
        #filter out small duration
        index_list = []
        for j in range(len(stay)):
            if depart_time[j] - arrive_time[j] + 1 > temporal_threshold:
                index_list.append(j)
    
        stay = np.asarray(stay)
        arrive_time = np.asarray(arrive_time)
        depart_time = np.asarray(depart_time)
    
        stay = stay[index_list]
        arrive_time = arrive_time[index_list]
        depart_time = depart_time[index_list]
        
        stay, arrive_time, depart_time = recalculate_stay(stay, arrive_time, depart_time, person_result, recalculate_threshold, spatial_threshold, num_node, time_length)
        
        stays.append(stay)
        arrive_times.append(arrive_time)
        depart_times.append(depart_time)
    return stays, arrive_times, depart_times

In [25]:
def extract_stay_without_time(route, spatial_threshold):
    #clusering method for reconsider set
    stay = []
    arrive_time = [0]
    depart_time = []
    
    prev = []
    for j in range(len(route)):
        current = route[j]
        if current >= 0:
            if max_distance(prev, current, num_node) > spatial_threshold:
                possible_location = np.bincount(prev).argmax()
                stay.append(possible_location)
                arrive_time.append(j)
                if j != 0:
                    depart_time.append(j - 1)
                prev = []

            prev.append(current)
    possible_location = np.bincount(prev).argmax()
    stay.append(possible_location)
    depart_time.append(j)
    
    stay = np.asarray(stay)
    arrive_time = np.asarray(arrive_time)
    depart_time = np.asarray(depart_time)
    duration = depart_time - arrive_time
    return stay, arrive_time, depart_time, duration
    

In [26]:
def calculate_distance(stay, origin, destination, num_node):
    #calculate distance for clustering method
    nodes = generate_node(num_node)
    num_stay = len(stay)
    result = np.zeros(num_stay)
    if origin == destination:
        for i in range(num_stay):
            result[i] = np.sqrt((nodes[origin, 0] - nodes[stay[i], 0])**2 + (nodes[origin, 1] - nodes[stay[i], 1])**2)
    else:
        if nodes[destination, 1] == nodes[origin, 1]:
            for i in range(num_stay):
                if (nodes[stay[i], 0] - nodes[origin, 0])*(nodes[stay[i], 0] - nodes[destination, 0]) < 0:
                    result[i] = np.abs(nodes[origin, 1] - nodes[stay[i], 1])
                else:
                    dis_origin = np.sqrt((nodes[origin, 0] - nodes[stay[i], 0])**2 + (nodes[origin, 1] - nodes[stay[i], 1])**2)
                    dis_destin = np.sqrt((nodes[destination, 0] - nodes[stay[i], 0])**2 + (nodes[destination, 1] - nodes[stay[i], 1])**2)
                    result[i] = np.min([dis_origin, dis_destin])                    
            
        elif nodes[destination, 0] == nodes[origin, 0]:
            for i in range(num_stay):
                if (nodes[stay[i], 1] - nodes[origin, 1])*(nodes[stay[i], 1] - nodes[destination, 1]) < 0:
                    result[i] = np.abs(nodes[origin, 0] - nodes[stay[i], 0])
                else:
                    dis_origin = np.sqrt((nodes[origin, 0] - nodes[stay[i], 0])**2 + (nodes[origin, 1] - nodes[stay[i], 1])**2)
                    dis_destin = np.sqrt((nodes[destination, 0] - nodes[stay[i], 0])**2 + (nodes[destination, 1] - nodes[stay[i], 1])**2)
                    result[i] = np.min([dis_origin, dis_destin]) 
        
        else:
            slope = (nodes[destination, 1] - nodes[origin, 1])/(nodes[destination, 0] - nodes[origin, 0])
            y_intercept = nodes[origin, 1] - slope * nodes[origin, 0]
            for i in range(num_stay):
                temp1 = - (nodes[stay[i], 0] - nodes[origin, 0])/slope + nodes[origin, 1] - nodes[stay[i], 1]
                temp2 = - (nodes[stay[i], 0] - nodes[destination, 0])/slope + nodes[destination, 1] - nodes[stay[i], 1]
                if temp1 * temp2  < 0:
                    result[i] = np.abs(slope * nodes[stay[i], 0] + y_intercept - nodes[stay[i], 1])/np.sqrt(1 + slope**2)
                else:
                    dis_origin = np.sqrt((nodes[origin, 0] - nodes[stay[i], 0])**2 + (nodes[origin, 1] - nodes[stay[i], 1])**2)
                    dis_destin = np.sqrt((nodes[destination, 0] - nodes[stay[i], 0])**2 + (nodes[destination, 1] - nodes[stay[i], 1])**2)
                    result[i] = np.min([dis_origin, dis_destin]) 
                    
    return result

In [27]:
def recalculate_stay(stay, arrive_time, depart_time, trajectory, recalculate_threshold, spatial_threshold, num_node, time_length):
    #including missed staying location 
    i = 0
    while i < len(stay) - 1:
        origin = stay[i]
        destination = stay[i + 1]
        start_time = depart_time[i]
        end_time = arrive_time[i + 1]
        route = trajectory[start_time : end_time + 1]
        
        if np.all(route < 0):
            i = i + 1
        else:
            temp_stay, temp_arrive_time, temp_depart_time, temp_duration = extract_stay_without_time(route, spatial_threshold)
            temp_arrive_time = temp_arrive_time + start_time
            temp_depart_time = temp_depart_time + start_time
            temp_distance = calculate_distance(temp_stay, origin, destination, num_node)
            temp_index = np.where(temp_distance > recalculate_threshold)
        
            if len(temp_index[0]) > 0:
                index = temp_index[0][temp_duration[temp_index].argmax()]
                new_stay = temp_stay[index]
                new_arrive_time = temp_arrive_time[index]
                new_depart_time = temp_depart_time[index]
            
                stay = np.insert(stay, i + 1, new_stay)
                arrive_time = np.insert(arrive_time, i + 1, new_arrive_time)
                depart_time = np.insert(depart_time, i + 1, new_depart_time)
            else:
                i = i + 1
    if depart_time[i] + 1 < time_length - 1:
        origin = stay[i]
        destination = stay[i]
        start_time = depart_time[i]
        route = trajectory[start_time :]
        temp_stay, temp_arrive_time, temp_depart_time, temp_duration = extract_stay_without_time(route, spatial_threshold)
        temp_arrive_time = temp_arrive_time + start_time
        temp_depart_time = temp_depart_time + start_time
        temp_distance = calculate_distance(temp_stay, origin, destination, num_node)
        temp_index = np.where(temp_distance > recalculate_threshold)

        if len(temp_index[0]) > 0:
            index = temp_index[0][temp_duration[temp_index].argmax()]
            new_stay = temp_stay[index]
            new_arrive_time = temp_arrive_time[index]
            new_depart_time = temp_depart_time[index]

            stay = np.append(stay, new_stay)
            arrive_time = np.append(arrive_time, new_arrive_time)
            depart_time = np.append(depart_time, new_depart_time)
            
    return stay, arrive_time, depart_time

In [28]:
def generate_cdr(multiple_percentage, empty_percentage, num_node, num_people, time_length, num_stay):
    #generate cdr with all properties
    observations = []
    stay_times = []
    for i in range(num_people):
        trajectory, stay, stay_time = generate_trajectory(time_length, num_node, num_stay)
        observation = generate_multi_observed(num_node, trajectory, population, multiple_percentage)
        observations.append(observation)
        stay_times.append(stay_time)
        if i == 0:
            trajectories = trajectory
            stays = stay
        else:
            trajectories = np.vstack((trajectories, trajectory))
            stays = np.vstack((stays, stay))
    
    time_stay = organize_stay_time(stays, stay_times, num_node)
    CDR = missing_observed(observations, empty_percentage)
    od = extract_od(stays, num_node)
    
    return od, CDR

In [29]:
def generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold):
    #calculate result with different method : conventional, hmm, hmm+modified
    result_hmm = real_viterbi(cdr, A, C, num_node)
    
    stay_obs, arrive_obs, depart_obs = extract_stay_obs(cdr, spatial_threshold, temporal_threshold)
    od_obs = extract_od(stay_obs, num_node)
    error_obs = np.sum(np.sum(np.abs(od_obs - od)))
    precision_obs = (250 * percentage_error(od_obs, od, num_node))/(2 * 250 * percentage_error(od_obs, od, num_node) + error_obs - 250)
    recall_obs = percentage_error(od_obs, od, num_node)
    
    stay_hmm, arrive_hmm, depart_hmm = extract_stay(result_hmm, spatial_threshold, temporal_threshold)
    od_hmm = extract_od(stay_hmm, num_node)
    error_hmm = np.sum(np.sum(np.abs(od_hmm - od)))
    precision_hmm = (250 * percentage_error(od_hmm, od, num_node))/(2 * 250 * percentage_error(od_hmm, od, num_node) + error_hmm - 250)
    recall_hmm = percentage_error(od_hmm, od, num_node)
    
    stay_hmm_modi, arrive_hmm_modi, depart_hmm_modi = extract_stay_modified(result_hmm, spatial_threshold, temporal_threshold, recalculate_threshold)
    od_hmm_modi = extract_od(stay_hmm_modi, num_node)
    error_hmm_modi = np.sum(np.sum(np.abs(od_hmm_modi - od)))
    precision_hmm_modi = (250 * percentage_error(od_hmm_modi, od, num_node))/(2 * 250 * percentage_error(od_hmm_modi, od, num_node) + error_hmm_modi - 250)
    recall_hmm_modi = percentage_error(od_hmm_modi, od, num_node)
    
    return precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi

In [30]:
A = np.load('A1_c.npy')
C = np.load('C1_c.npy')
# 10% multi, 50 empty
r_p_obs_1 = np.zeros((6, 20))
r_r_obs_1 = np.zeros((6, 20))
r_p_hmm_1 = np.zeros((6, 20))
r_r_hmm_1 = np.zeros((6, 20))
r_p_modi_1 = np.zeros((6, 20))
r_r_modi_1 = np.zeros((6, 20))
for i in range(20):
    od, cdr = generate_cdr(0.1, 0.5, num_node, num_people, time_length, num_stay)
    
    spatial_threshold = 1.5
    temporal_threshold = 3
    recalculate_threshold = 2
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_1[0, i] = precision_obs
    r_r_obs_1[0, i] = recall_obs
    r_p_hmm_1[0, i] = precision_hmm
    r_r_hmm_1[0, i] = recall_hmm
    r_p_modi_1[0, i] = precision_hmm_modi
    r_r_modi_1[0, i] = recall_hmm_modi
    
#     spatial_threshold = 1.5
#     temporal_threshold = 6
#     recalculate_threshold = 2
#     precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
#     r_p_obs_1[1, i] = precision_obs
#     r_r_obs_1[1, i] = recall_obs
#     r_p_hmm_1[1, i] = precision_hmm
#     r_r_hmm_1[1, i] = recall_hmm
#     r_p_modi_1[1, i] = precision_hmm_modi
#     r_r_modi_1[1, i] = recall_hmm_modi
    
    spatial_threshold = 2.5
    temporal_threshold = 6
    recalculate_threshold = 2
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_1[2, i] = precision_obs
    r_r_obs_1[2, i] = recall_obs
    r_p_hmm_1[2, i] = precision_hmm
    r_r_hmm_1[2, i] = recall_hmm
    r_p_modi_1[2, i] = precision_hmm_modi
    r_r_modi_1[2, i] = recall_hmm_modi
    
    spatial_threshold = 1.5
    temporal_threshold = 3
    recalculate_threshold = 3
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_1[3, i] = precision_obs
    r_r_obs_1[3, i] = recall_obs
    r_p_hmm_1[3, i] = precision_hmm
    r_r_hmm_1[3, i] = recall_hmm
    r_p_modi_1[3, i] = precision_hmm_modi
    r_r_modi_1[3, i] = recall_hmm_modi
    
#     spatial_threshold = 1.5
#     temporal_threshold = 6
#     recalculate_threshold = 3
#     precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
#     r_p_obs_1[4, i] = precision_obs
#     r_r_obs_1[4, i] = recall_obs
#     r_p_hmm_1[4, i] = precision_hmm
#     r_r_hmm_1[4, i] = recall_hmm
#     r_p_modi_1[4, i] = precision_hmm_modi
#     r_r_modi_1[4, i] = recall_hmm_modi
    
    spatial_threshold = 2.5
    temporal_threshold = 6
    recalculate_threshold = 3
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_1[5, i] = precision_obs
    r_r_obs_1[5, i] = recall_obs
    r_p_hmm_1[5, i] = precision_hmm
    r_r_hmm_1[5, i] = recall_hmm
    r_p_modi_1[5, i] = precision_hmm_modi
    r_r_modi_1[5, i] = recall_hmm_modi

In [31]:
np.save('r_p_obs_1c.npy', r_p_obs_1)
np.save('r_r_obs_1c.npy', r_r_obs_1)
np.save('r_p_hmm_1c.npy', r_p_hmm_1)
np.save('r_r_hmm_1c.npy', r_r_hmm_1)
np.save('r_p_modi_1c.npy', r_p_modi_1)
np.save('r_r_modi_1c.npy', r_r_modi_1)
r_p_obs1 = np.average(r_p_obs_1, 1)
r_r_obs1 = np.average(r_r_obs_1, 1)
r_p_hmm1 = np.average(r_p_hmm_1, 1)
r_r_hmm1 = np.average(r_r_hmm_1, 1)
r_p_modi1 = np.average(r_p_modi_1, 1)
r_r_modi1 = np.average(r_r_modi_1, 1)
print(r_p_obs1*100)
print(r_r_obs1*100)
print(r_p_hmm1*100)
print(r_r_hmm1*100)
print(r_p_modi1*100)
print(r_r_modi1*100)

[50.5136174   0.         66.93108414 50.5136174   0.         66.93108414]
[57.8  0.  46.8 57.8  0.  46.8]
[71.64527595  0.         67.76337941 71.64527595  0.         67.76337941]
[73.34  0.   45.82 73.34  0.   45.82]
[71.34088355  0.         70.4186677  71.50990012  0.         69.62952479]
[74.46  0.   60.2  74.14  0.   56.74]


In [32]:
A = np.load('A2_c.npy')
C = np.load('C2_c.npy')
# 30% multi, 30 empty
r_p_obs_2 = np.zeros((6, 20))
r_r_obs_2 = np.zeros((6, 20))
r_p_hmm_2 = np.zeros((6, 20))
r_r_hmm_2 = np.zeros((6, 20))
r_p_modi_2 = np.zeros((6, 20))
r_r_modi_2 = np.zeros((6, 20))
for i in range(20):
    od, cdr = generate_cdr(0.3, 0.3, num_node, num_people, time_length, num_stay)
    
    spatial_threshold = 1.5
    temporal_threshold = 3
    recalculate_threshold = 2
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_2[0, i] = precision_obs
    r_r_obs_2[0, i] = recall_obs
    r_p_hmm_2[0, i] = precision_hmm
    r_r_hmm_2[0, i] = recall_hmm
    r_p_modi_2[0, i] = precision_hmm_modi
    r_r_modi_2[0, i] = recall_hmm_modi
    
#     spatial_threshold = 1.5
#     temporal_threshold = 6
#     recalculate_threshold = 2
#     precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
#     r_p_obs_2[1, i] = precision_obs
#     r_r_obs_2[1, i] = recall_obs
#     r_p_hmm_2[1, i] = precision_hmm
#     r_r_hmm_2[1, i] = recall_hmm
#     r_p_modi_2[1, i] = precision_hmm_modi
#     r_r_modi_2[1, i] = recall_hmm_modi
    
    spatial_threshold = 2.5
    temporal_threshold = 6
    recalculate_threshold = 2
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_2[2, i] = precision_obs
    r_r_obs_2[2, i] = recall_obs
    r_p_hmm_2[2, i] = precision_hmm
    r_r_hmm_2[2, i] = recall_hmm
    r_p_modi_2[2, i] = precision_hmm_modi
    r_r_modi_2[2, i] = recall_hmm_modi
    
    spatial_threshold = 1.5
    temporal_threshold = 3
    recalculate_threshold = 3
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_2[3, i] = precision_obs
    r_r_obs_2[3, i] = recall_obs
    r_p_hmm_2[3, i] = precision_hmm
    r_r_hmm_2[3, i] = recall_hmm
    r_p_modi_2[3, i] = precision_hmm_modi
    r_r_modi_2[3, i] = recall_hmm_modi
    
#     spatial_threshold = 1.5
#     temporal_threshold = 6
#     recalculate_threshold = 3
#     precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
#     r_p_obs_2[4, i] = precision_obs
#     r_r_obs_2[4, i] = recall_obs
#     r_p_hmm_2[4, i] = precision_hmm
#     r_r_hmm_2[4, i] = recall_hmm
#     r_p_modi_2[4, i] = precision_hmm_modi
#     r_r_modi_2[4, i] = recall_hmm_modi
    
    spatial_threshold = 2.5
    temporal_threshold = 6
    recalculate_threshold = 3
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_2[5, i] = precision_obs
    r_r_obs_2[5, i] = recall_obs
    r_p_hmm_2[5, i] = precision_hmm
    r_r_hmm_2[5, i] = recall_hmm
    r_p_modi_2[5, i] = precision_hmm_modi
    r_r_modi_2[5, i] = recall_hmm_modi

In [33]:
np.save('r_p_obs_2c.npy', r_p_obs_2)
np.save('r_r_obs_2c.npy', r_r_obs_2)
np.save('r_p_hmm_2c.npy', r_p_hmm_2)
np.save('r_r_hmm_2c.npy', r_r_hmm_2)
np.save('r_p_modi_2c.npy', r_p_modi_2)
np.save('r_r_modi_2c.npy', r_r_modi_2)
r_p_obs2 = np.average(r_p_obs_2, 1)
r_r_obs2 = np.average(r_r_obs_2, 1)
r_p_hmm2 = np.average(r_p_hmm_2, 1)
r_r_hmm2 = np.average(r_r_hmm_2, 1)
r_p_modi2 = np.average(r_p_modi_2, 1)
r_r_modi2 = np.average(r_r_modi_2, 1)
print(r_p_obs2*100)
print(r_r_obs2*100)
print(r_p_hmm2*100)
print(r_r_hmm2*100)
print(r_p_modi2*100)
print(r_r_modi2*100)

[63.2541273   0.         74.25690931 63.2541273   0.         74.25690931]
[69.54  0.   51.46 69.54  0.   51.46]
[81.00289859  0.         73.94960977 81.00289859  0.         73.94960977]
[82.5   0.   50.76 82.5   0.   50.76]
[80.69450034  0.         77.74415822 81.04700656  0.         76.77831144]
[84.1   0.   67.28 83.76  0.   63.26]


In [34]:
A = np.load('A3_c.npy')
C = np.load('C3_c.npy')
# 50% multi, 10 empty
r_p_obs_3 = np.zeros((6, 20))
r_r_obs_3 = np.zeros((6, 20))
r_p_hmm_3 = np.zeros((6, 20))
r_r_hmm_3 = np.zeros((6, 20))
r_p_modi_3 = np.zeros((6, 20))
r_r_modi_3 = np.zeros((6, 20))
for i in range(20):
    od, cdr = generate_cdr(0.5, 0.1, num_node, num_people, time_length, num_stay)
    
    spatial_threshold = 1.5
    temporal_threshold = 3
    recalculate_threshold = 2
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_3[0, i] = precision_obs
    r_r_obs_3[0, i] = recall_obs
    r_p_hmm_3[0, i] = precision_hmm
    r_r_hmm_3[0, i] = recall_hmm
    r_p_modi_3[0, i] = precision_hmm_modi
    r_r_modi_3[0, i] = recall_hmm_modi
    
#     spatial_threshold = 1.5
#     temporal_threshold = 6
#     recalculate_threshold = 2
#     precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
#     r_p_obs_3[1, i] = precision_obs
#     r_r_obs_3[1, i] = recall_obs
#     r_p_hmm_3[1, i] = precision_hmm
#     r_r_hmm_3[1, i] = recall_hmm
#     r_p_modi_3[1, i] = precision_hmm_modi
#     r_r_modi_3[1, i] = recall_hmm_modi
    
    spatial_threshold = 2.5
    temporal_threshold = 6
    recalculate_threshold = 2
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_3[2, i] = precision_obs
    r_r_obs_3[2, i] = recall_obs
    r_p_hmm_3[2, i] = precision_hmm
    r_r_hmm_3[2, i] = recall_hmm
    r_p_modi_3[2, i] = precision_hmm_modi
    r_r_modi_3[2, i] = recall_hmm_modi
    
    spatial_threshold = 1.5
    temporal_threshold = 3
    recalculate_threshold = 3
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_3[3, i] = precision_obs
    r_r_obs_3[3, i] = recall_obs
    r_p_hmm_3[3, i] = precision_hmm
    r_r_hmm_3[3, i] = recall_hmm
    r_p_modi_3[3, i] = precision_hmm_modi
    r_r_modi_3[3, i] = recall_hmm_modi
    
#     spatial_threshold = 1.5
#     temporal_threshold = 6
#     recalculate_threshold = 3
#     precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
#     r_p_obs_3[4, i] = precision_obs
#     r_r_obs_3[4, i] = recall_obs
#     r_p_hmm_3[4, i] = precision_hmm
#     r_r_hmm_3[4, i] = recall_hmm
#     r_p_modi_3[4, i] = precision_hmm_modi
#     r_r_modi_3[4, i] = recall_hmm_modi
    
    spatial_threshold = 2.5
    temporal_threshold = 6
    recalculate_threshold = 3
    precision_obs, recall_obs, precision_hmm, recall_hmm, precision_hmm_modi, recall_hmm_modi = generate_result(cdr, od, A, C, num_node, spatial_threshold, temporal_threshold, recalculate_threshold)
    
    r_p_obs_3[5, i] = precision_obs
    r_r_obs_3[5, i] = recall_obs
    r_p_hmm_3[5, i] = precision_hmm
    r_r_hmm_3[5, i] = recall_hmm
    r_p_modi_3[5, i] = precision_hmm_modi
    r_r_modi_3[5, i] = recall_hmm_modi

In [35]:
np.save('r_p_obs_3c.npy', r_p_obs_3)
np.save('r_r_obs_3c.npy', r_r_obs_3)
np.save('r_p_hmm_3c.npy', r_p_hmm_3)
np.save('r_r_hmm_3c.npy', r_r_hmm_3)
np.save('r_p_modi_3c.npy', r_p_modi_3)
np.save('r_r_modi_3c.npy', r_r_modi_3)
r_p_obs3 = np.average(r_p_obs_3, 1)
r_r_obs3 = np.average(r_r_obs_3, 1)
r_p_hmm3 = np.average(r_p_hmm_3, 1)
r_r_hmm3 = np.average(r_r_hmm_3, 1)
r_p_modi3 = np.average(r_p_modi_3, 1)
r_r_modi3 = np.average(r_r_modi_3, 1)
print(r_p_obs3*100)
print(r_r_obs3*100)
print(r_p_hmm3*100)
print(r_r_hmm3*100)
print(r_p_modi3*100)
print(r_r_modi3*100)

[78.37690075  0.         78.8024523  78.37690075  0.         78.8024523 ]
[79.96  0.   55.18 79.96  0.   55.18]
[89.59193834  0.         78.85127599 89.59193834  0.         78.85127599]
[90.2   0.   55.18 90.2   0.   55.18]
[89.06117895  0.         83.80271227 89.53233948  0.         82.35123536]
[91.24  0.   73.26 91.18  0.   68.74]
